In [26]:
pip install praw python-dotenv requests transformers torch pandas


Note: you may need to restart the kernel to use updated packages.


In [27]:
import praw
from dotenv import load_dotenv
import os

load_dotenv()

def scrape_reddit_user(url):
    reddit = praw.Reddit(
        client_id=os.getenv("REDDIT_CLIENT_ID"),
        client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
        user_agent=os.getenv("REDDIT_USER_AGENT")
    )
    
    username = url.split("/user/")[-1].strip("/")
    user = reddit.redditor(username)
    
    comments = []
    posts = []
    
    # Scrape recent comments (limit: 100)
    for comment in user.comments.new(limit=100):
        comments.append(comment.body)
    
    # Scrape recent posts (limit: 50)
    for submission in user.submissions.new(limit=50):
        posts.append(submission.title + " " + (submission.selftext if submission.selftext else ""))
    
    return {"comments": comments, "posts": posts}

    

In [28]:
import requests

def generate_persona_groq(text_data):
    API_URL = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": "Bearer YOUR_GROQ_API_KEY",  # Get free key: https://console.groq.com/
        "Content-Type": "application/json"
    }
    
    prompt = f"""
    Analyze these Reddit posts/comments and generate a detailed User Persona.
    Include demographics, interests, and personality traits (like Big Five or MBTI).
    For each trait, cite the exact post/comment that supports it.

    Example Output:
    - **Demographic**: Male, 25-30 (from comment: "As a guy in tech...").
    - **Interests**: AI, Python (from post: "Built a Llama-3 fine-tuner").
    - **Personality**: High Openness (from comment: "I love exploring new ideas").

    Input:
    {text_data}
    """
    
    data = {
        "messages": [{"role": "user", "content": prompt}],
        "model": "llama3-8b-8192"  # or "llama3-70b-8192"
    }
    
    response = requests.post(API_URL, headers=headers, json=data)
    return response.json()["choices"][0]["message"]["content"]

In [29]:
def save_persona_to_file(username, persona_text):
    with open(f"{username}_persona.txt", "w") as f:
        f.write(persona_text)

In [30]:
def save_persona_to_file(username, persona_text):
    with open(f"{username}_persona.txt", "w") as f:
        f.write(persona_text)

def generate_persona(text):
    # Example: just return the first 300 characters
    return f"Persona summary based on text:\n{text[:300]}"

if __name__ == "__main__":
    reddit_url = input("Enter Reddit Profile URL: ")

    # Step 1: Scrape Data
    user_data = scrape_reddit_user(reddit_url)
    combined_text = "\n".join(user_data["comments"] + user_data["posts"])

    # Step 2: Generate Persona
    persona = generate_persona(combined_text)

    # Step 3: Save to File
    username = reddit_url.split("/user/")[-1].strip("/")
    save_persona_to_file(username, persona)

    print(f"Persona saved to {username}_persona.txt")

Persona saved to Hungry-Move-6603_persona.txt


In [31]:
import os
print(os.getcwd())  # Shows where the file was saved

/workspaces/Gen-AI---Personlity-Analyser/experiment
